In [ ]:
import random
import io

import keras
import KerasTools as KT
import numpy as np
import tensorflow as tf

import PIL
import bqplot.pyplot
import ipywidgets

tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()
mapping = KT.datasets.decode._mapping['cifar10']
labels = ['Automobile', 'Bird', 'Cat', 'Horse']

#(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar100.load_data(label_mode='fine')
#mapping = KT.datasets.decode._mapping['cifar100_fine']
#labels = ['Roses', 'Sunflowers']

cifar_map = dict()
random.seed(0)

train_pairs = []
test_pairs = []
for i,l in enumerate(labels):
    idx = mapping.index(l)
    cifar_map[i] = l
    idx_train = train_images[train_labels.flatten() == idx]
    idx_test = test_images[test_labels.flatten() == idx]
    train_pairs.extend([(img,i) for img in idx_train])
    test_pairs.extend([(img,i) for img in idx_test])

random.shuffle(train_pairs)
random.shuffle(test_pairs)

train_images, train_labels = zip(*train_pairs)
test_images, test_labels = zip(*test_pairs)
train_images = np.asarray(train_images)
train_labels = np.asarray(train_labels)
test_images = np.asarray(test_images)
test_labels = np.asarray(test_labels)
print(cifar_map)
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

In [ ]:
r, c, s = 6, 8, 128
pics = train_images[np.random.randint(train_images.shape[0], size=r*c)]
canvas = PIL.Image.new('RGB', (c*s+2, r*s+2), color='white')
for i,d in enumerate(pics):
    dimg = PIL.Image.fromarray(d).resize((s-8, s-8))
    canvas.paste(dimg, box=(s*int(i/r), s*(i%r)))

buf = io.BytesIO()
canvas.save(buf, 'gif')
img = ipywidgets.Image(value=buf.getvalue())
display(img)

In [ ]:
x_train = train_images.astype('float32') / 255
x_test = test_images.astype('float32') / 255

y_train = keras.utils.to_categorical(train_labels)
y_test = keras.utils.to_categorical(test_labels)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)


In [ ]:
def init_plots():
    axes_loss = {'x': {'label': 'Epochs'}, 
                 'y': {'label': 'Losses', 
                       'label_offset': '50px',
                       'tick_style': {'font-size': 10}}}
    axes_acc = {'x': {'label': 'Epochs'}, 
                'y': {'label': 'Accuracy', 
                      'label_offset': '50px',
                       'tick_style': {'font-size': 10}}}
        
    loss_plt = bqplot.pyplot.figure(min_aspect_ratio=4/3, max_aspect_ratio=4/3)
    bqplot.pyplot.plot([0,1],[0.5,0.5], axes_options=axes_loss)
    bqplot.pyplot.plot([0,1],[0.75,0.75], colors=['orange'])
    acc_plt  = bqplot.pyplot.figure(min_aspect_ratio=4/3, max_aspect_ratio=4/3)
    bqplot.pyplot.plot([0,1],[0.5,0.5], axes_options=axes_acc)
    bqplot.pyplot.plot([0,1],[0.75,0.75], colors=['orange'])        

    return (loss_plt, acc_plt)

class plot_history(keras.callbacks.Callback):
    def __init__(self, loss_plt, acc_plt):
        self.loss_plt = loss_plt
        self.acc_plt  = acc_plt
        self.history = {'loss':[], 'val_loss':[], 'acc': [], 'val_acc':[]}

    def on_epoch_end(self, epoch, logs={}):
        self.history['loss'].append(logs.get('loss'))
        self.history['val_loss'].append(logs.get('val_loss'))
        self.history['acc'].append(logs.get('acc'))
        self.history['val_acc'].append(logs.get('val_acc'))

        x_data = range(1, len(self.history['loss'])+1)
        self.loss_plt.marks[0].x = x_data
        self.loss_plt.marks[0].y = self.history['loss']
        self.loss_plt.marks[1].x = x_data
        self.loss_plt.marks[1].y = self.history['val_loss']
        
        x_data = range(1, len(self.history['acc'])+1)
        self.acc_plt.marks[0].x = x_data
        self.acc_plt.marks[0].y = self.history['acc']
        self.acc_plt.marks[1].x = x_data
        self.acc_plt.marks[1].y = self.history['val_acc']

In [ ]:
def build_network():
    network = keras.models.Sequential(name='CIFAR_CNN')
    network.add(keras.layers.Conv2D(8, 3, padding='same', activation='relu', input_shape=(32,32,3), name='conv1a'))
    network.add(keras.layers.MaxPooling2D(2, name='pool1'))
    network.add(keras.layers.Conv2D(16, 3, padding='same', activation='relu', name='conv2a'))
    network.add(keras.layers.MaxPooling2D(2, name='pool2'))
    network.add(keras.layers.Conv2D(32, 3, padding='same', activation='relu', name='conv3a'))
    network.add(keras.layers.MaxPooling2D(2, name='pool3'))
    network.add(keras.layers.Conv2D(64, 3, padding='same', activation='relu', name='conv4a'))
    network.add(keras.layers.GlobalMaxPooling2D(name='globalpool'))
    network.add(keras.layers.Dropout(0.25, name='dropout'))
    network.add(keras.layers.Dense(128, activation='relu', name='dense1'))
    network.add(keras.layers.Dense(len(cifar_map), activation='softmax', name='output'))

    network.compile(optimizer=keras.optimizers.rmsprop(), loss='categorical_crossentropy', metrics=['accuracy'])

    return network

In [ ]:
network = build_network()
network.summary()

loss_plt, acc_plt = init_plots()
display(ipywidgets.HBox([loss_plt, acc_plt]))

In [ ]:
%%time
traingen = keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True, 
    fill_mode='nearest')
traingen.fit(train_images)

valgen = keras.preprocessing.image.ImageDataGenerator()

train_generator = traingen.flow(x_train, y_train, batch_size=32)
val_generator = valgen.flow(x_test, y_test, batch_size=32)
epochs = 50
history = network.fit_generator(train_generator, validation_data=val_generator, 
                                use_multiprocessing=True, workers=4,
                                epochs=epochs, verbose=0, 
                                callbacks=[plot_history(loss_plt, acc_plt)])

In [ ]:
entry = 4

prediction = network.predict(test_images[entry:entry+1])
ground_truth = np.argmax(y_test[entry])
prediction_label = np.argmax(prediction[0])

img = test_images[entry]
buf = io.BytesIO()
PIL.Image.fromarray(img).resize((192, 192)).save(buf, 'gif')
img = ipywidgets.Image(value=buf.getvalue())
display(img)

print("Ground truth: {} / {}".format(ground_truth, cifar_map[ground_truth]))
print("Prediction:   {} / {}".format(prediction_label, cifar_map[prediction_label]))
print('-' * 26)
print("{0:<10} | {1:>6} ".format("Label", "Score"))
print('-' * 26)
for i,r in enumerate(prediction[0]):
    print("{:<10} | {:> 8.2%}".format(cifar_map[i], r))
print('-' * 26)

